In [14]:
import pandas as pd
import sqlite3
import zipfile
import tempfile
import os

zip_path = 'mock_resq.db.zip'
db_name = 'mock_resq.db'


with tempfile.TemporaryDirectory() as tmp_dir:
    # Extract the SQLite database from the zip file
    with zipfile.ZipFile(zip_path, 'r') as zip_ref:
        zip_ref.extractall(tmp_dir)
    db_path = os.path.join(tmp_dir, db_name)
    conn = sqlite3.connect(db_path)


# Problem 1

In [15]:
query = """
SELECT
    orders.'providerId',
    SUM(orders.sales) AS sales_total
FROM orders
GROUP BY providerId
ORDER BY SUM(sales) DESC
LIMIT 10;
"""
df = pd.read_sql_query(sql=query, con=conn)
df

,providerId,sales_total
0,7198110370745783236,10917800
1,8312310143652755348,7467750
2,8097235958083241788,2383700
3,3865474760205653333,2223400
4,8084884958338058541,1868140
5,4734853230275691017,1702100
6,5305286819167536850,1690500
7,1066258454353124935,1568100
8,7642201963087705313,1472000
9,4014236829817167297,1457000


In [16]:
query = """
SELECT 
    providers.'defaultOfferType' AS offer_type,
    COUNT(orders.'id') AS count_orders
FROM orders
LEFT JOIN providers ON providers.'id' = orders.'providerId'
GROUP BY offer_type
ORDER BY count_orders DESC;
"""
df = pd.read_sql_query(sql=query, con=conn)
df

,offer_type,count_orders
0,meal,219764
1,snack,49861
2,grocery-bag,27192
3,dessert,1910
4,ingredients,1091
5,flowers,153


In [20]:
query ="""
DROP TABLE IF EXISTS tempOrder;
CREATE TEMP TABLE tempOrder AS
SELECT *,
       ROW_NUMBER() OVER (PARTITION BY userId ORDER BY createdAt) AS order_row
FROM orders;

UPDATE tempOrder
SET createdAt = DATE(createdAt);



DROP TABLE IF EXISTS tempOrderCount;
CREATE TEMP TABLE tempOrderCount AS
SELECT *,
	MIN(createdAt) OVER (PARTITION BY userId) AS first_order_date
FROM tempOrder;


DROP TABLE IF EXISTS tempOrder;
CREATE TEMP TABLE tempOrder AS
SELECT *,
	JULIANDAY(createdAt) - JULIANDAY(first_order_date) AS days_difference 
FROM tempOrderCount
WHERE (order_row = 2);

DROP TABLE IF EXISTS userRetention;


CREATE TEMP TABLE userRetention AS
SELECT userId,
	(
    CASE
      WHEN (days_difference <= 30) THEN 'M0'
      WHEN (days_difference > 30) AND (days_difference < 62) THEN 'M1'
      WHEN (days_difference <= 30) THEN 'M0'
      ELSE 'M2+'
    END
	) AS retention
FROM tempOrder;

"""

try:
    cur = conn.cursor()
    cur.executescript(query)
    conn.commit()
    print("Customer Retention classification completed.")


    df = pd.read_sql_query(sql="SELECT * FROM userRetention", con=conn)
    print(df)

except sqlite3.Error as e:
    print(f"An error occurred: {e}")
    

Customer Retention classification completed.
                    userId retention
0          219034330643057       M2+
1          406087302631582        M0
2          472654236140424       M2+
3          494869758008189       M2+
4          857319303011182        M1
...                    ...       ...
56742  9223090445069821952        M1
56743  9223097207332177627        M1
56744  9223113765943846674       M2+
56745  9223115497170789549        M0
56746  9223201399781576886       M2+

[56747 rows x 2 columns]


# Problem 2

In [29]:
users = pd.read_sql_query(sql="SELECT * FROM users;", con=conn)
users.registeredDate = pd.to_datetime(users.registeredDate)
users.country = users.country.astype(dtype="category")
users = users.rename(columns={"id": "userId"})


orders = pd.read_sql_query(sql="SELECT * FROM orders", con=conn)
orders.createdAt = pd.to_datetime(orders.createdAt)
for col in ["quantity", "refunded", "sales"]:
    orders[col] = pd.to_numeric(orders[col])

orders = orders.rename(columns={"id": "orderId"})
orders = orders.merge(users, on="userId", how="left")

orders["t1"]  = (orders.createdAt - orders.registeredDate).dt.days.astype(int)
orders = orders.sort_values(["userId", "t1"]).reset_index(drop=True)
orders["t0"] = orders.groupby(["userId"]).t1.shift(1).fillna(0).astype(int)
orders["duration"] = orders["t1"] - orders["t0"]
orders["order_no"] = orders.groupby(["userId"]).cumcount() + 1

providers = pd.read_sql_query(sql="SELECT * FROM providers", con=conn).rename(columns={
    "id":"providerId", 
    "country":"providerCountry"
    })
providers.defaultOfferType = providers.defaultOfferType.astype("category")
orders = orders.merge(providers[["providerId", "defaultOfferType","providerCountry"]], on = "providerId", how="left")

In [36]:
APV = (
    orders
    .groupby(
        by=["providerCountry","defaultOfferType","currency"], 
        observed=True
        )
    .agg(func={
        "sales":"sum",
        "orderId":"count"
        })
    .assign(APV = lambda x: (x.sales / x.orderId).round(3))
    [["APV"]]
    .reset_index()
)
APV

,providerCountry,defaultOfferType,currency,APV
0,deu,meal,eur,334.615
1,est,grocery-bag,eur,548.285
2,est,meal,eur,556.269
3,est,snack,eur,389.220
4,fin,dessert,eur,688.691
5,fin,flowers,eur,384.869
6,fin,grocery-bag,eur,565.215
7,fin,ingredients,eur,676.951
8,fin,meal,eur,667.768
9,fin,snack,eur,486.086


In [37]:
APFR = (
    orders
    .groupby(
        by=["providerCountry","defaultOfferType","currency"], 
        observed=True
        )
    .agg(func={
            "userId":"nunique",
            "orderId":"count"
        })
    .assign(APFR = lambda x: (x.orderId / x.userId).round(3)) 
    [["APFR"]]
    .reset_index()
)
APFR

,providerCountry,defaultOfferType,currency,APFR
0,deu,meal,eur,2.294
1,est,grocery-bag,eur,1.522
2,est,meal,eur,2.740
3,est,snack,eur,1.319
4,fin,dessert,eur,1.107
5,fin,flowers,eur,1.159
6,fin,grocery-bag,eur,2.161
7,fin,ingredients,eur,1.364
8,fin,meal,eur,2.185
9,fin,snack,eur,1.653


In [39]:
CV = (
    APV
    .merge(
        right=APFR, 
        on=["providerCountry","defaultOfferType","currency"], 
        how="inner"
        )
    .assign(CV = lambda x: (x.APV / x.APFR).round(3))
    .drop(columns=["APFR", "APV"])
)
CV

,providerCountry,defaultOfferType,currency,CV
0,deu,meal,eur,145.865
1,est,grocery-bag,eur,360.240
2,est,meal,eur,203.018
3,est,snack,eur,295.087
4,fin,dessert,eur,622.124
5,fin,flowers,eur,332.070
6,fin,grocery-bag,eur,261.553
7,fin,ingredients,eur,496.298
8,fin,meal,eur,305.615
9,fin,snack,eur,294.063


In [41]:
ACL = (
    orders
    .groupby(
        by=["userId","providerCountry","currency"], 
        observed=True
        )
    .agg(func={
        "defaultOfferType":"first",
        "t1":"max"
    })
    .reset_index()
    .groupby(
        by=["providerCountry","defaultOfferType","currency"], 
        observed=True
        )
    .agg({"t1": "sum", "userId": "count"})
    .assign(ACL = lambda x:(x.t1 / x.userId).round(3))
    [["ACL"]]
    .reset_index()
)
ACL

,providerCountry,defaultOfferType,currency,ACL
0,deu,meal,eur,724.765
1,est,grocery-bag,eur,220.753
2,est,meal,eur,280.535
3,est,snack,eur,200.900
4,fin,dessert,eur,929.058
5,fin,flowers,eur,716.211
6,fin,grocery-bag,eur,886.921
7,fin,ingredients,eur,936.456
8,fin,meal,eur,942.536
9,fin,snack,eur,996.547
